In [1]:
from time import perf_counter as now

from rest import build_tempo_spatial_index, base_query
from utilities import dataset
from utilities.box2D import Box2D
from utilities.config import config
from utilities.data_preprocessing import traj_data, gen_border

In [2]:
import os
import dill

def get_index(file_name, grid_space):
    base = os.path.basename(file_name).split('.')
    traj_fname = f'resource/{base[0]}_traj.pkl'
    if os.path.exists(traj_fname):
        with open(traj_fname, 'rb') as f:
            trajs = dill.load(f)
    else:
        data, cols, cls_map = dataset.load_yolo_for(file_name)
        trajs = traj_data(data, cols, 5, cls_map, scale=1)
        with open(traj_fname, 'wb') as f:
            dill.dump(trajs, f)
    broders = gen_border(trajs.bbox, *grid_space)
    config.gird_border = broders
    temp_spt = build_tempo_spatial_index(trajs)
    return temp_spt

filename = '/home/lg/VDBM/spatiotemporal/resource/dataset/traj_taipei_0412.pkl'

In [ ]:
tempo_spat = get_index(filename, (8,6))

In [3]:
tempo_spat = get_index(filename, (2,2))

In [4]:
tempo_spat[0]._outer._outer.borders

(array([   0, 1275, 1276]), array([  0, 712, 713]))

In [5]:
label = {0:1}
interval = 0, 60*60*30
duration = 10, 100

In [6]:
region = Box2D((546, 727, 427, 569))
count = 0
start = now()
for _ in base_query(tempo_spat, region, label, duration, interval):
    count += 1
end = now()

In [356]:
region = Box2D((3277, 3553, 1426, 1667))
# region = Box2D((0, 3840, 0, 2160))
# label = {0: 1, 2: 1, 1: 1, 3: 1, 5: 1, 7: 1}


start = now()
for _ in base_query(tempo_spat, region, label, duration, interval):
    count += 1
end = now()
print('result count:', count, 'using', end - start, 's')

result count: 531 using 0.3478171930000826 s


In [7]:
print('result count:', count, 'using', end - start, 's')

result count: 5 using 0.3333193680000477 s
